In [4]:
# Testing use case https://arxiv.org/pdf/2103.00903.pdf 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, Flatten, Activation, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

from tensorflow.keras.datasets import mnist
import keras

# Generate the train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)
 
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))
 
# Generate some noisy data
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
 
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [5]:
import tensorflow as tf
import numpy as np

def weights_diff(model1, model2):
    model_diff = []
    layer_sizes = []
    for i in range(len(model1.layers)):
      if len(model1.layers[i].get_weights()) > 0:
#         print("Layer " + str(i + 1) + ":")
        layer_diff = model1.layers[i].get_weights()[0] - model2.layers[i].get_weights()[0]
        model_diff.append(layer_diff)
#         print(layer_diff)
    for i in range(len(model_diff)):
      current_layer_size = 0
      total_nonzero = 0
      max = 0
      for cell in np.nditer(model_diff[i]):
        current_layer_size += 1
        if abs(cell) > 0.01:
          total_nonzero += 1
          if abs(cell) > max:
            max = cell
      percentage_diff = ((total_nonzero * 1.) / current_layer_size) * 100
      print("Amount of different weights in Layer " + str(i + 1) + ": " + str(total_nonzero)
            + " / " + str(current_layer_size) + " (" + str(percentage_diff) + "%)")
      print("Maximum Difference in Layer " + str(i+1) + ": " + str(max))
      layer_sizes.append(current_layer_size)

        
def weights_diff_stats(model1, model2):
    model_diff = []
    layer_sizes = []
    for i in range(len(model1.layers)):
      if len(model1.layers[i].get_weights()) > 0:
#         print("Layer " + str(i + 1) + ":")
        layer_diff = model1.layers[i].get_weights()[0] - model2.layers[i].get_weights()[0]
        model_diff.append(layer_diff)
#         print(layer_diff)
    res = []
    for i in range(len(model_diff)):
      current_layer_size = 0
      total_nonzero = 0
      max = 0
      for cell in np.nditer(model_diff[i]):
        current_layer_size += 1
        if abs(cell) > 0.01:
          total_nonzero += 1
          if abs(cell) > max:
            max = cell
      percentage_diff = ((total_nonzero * 1.) / current_layer_size) * 100
      res.append(percentage_diff)
#       print("Amount of different weights in Layer " + str(i + 1) + ": " + str(total_nonzero)
#             + " / " + str(current_layer_size) + " (" + str(percentage_diff) + "%)")
#       print("Maximum Difference in Layer " + str(i+1) + ": " + str(max))
      layer_sizes.append(current_layer_size)
    return res

In [11]:
m1 = tf.keras.models.load_model('redundancy_check/teacher_old_60')
m2 = tf.keras.models.load_model('redundancy_check/student_old_60')
weights_diff(m1, m2)

Amount of different weights in Layer 1: 297 / 324 (91.66666666666666%)
Maximum Difference in Layer 1: 0.51335806
Amount of different weights in Layer 2: 11254 / 11664 (96.4849108367627%)
Maximum Difference in Layer 2: 1.1523159
Amount of different weights in Layer 3: 11279 / 11664 (96.69924554183812%)
Maximum Difference in Layer 3: 1.2177842
Amount of different weights in Layer 4: 11288 / 11664 (96.7764060356653%)
Maximum Difference in Layer 4: 0.91574544
Amount of different weights in Layer 5: 873658 / 903168 (96.73261231575964%)
Maximum Difference in Layer 5: 1.2563202
Amount of different weights in Layer 6: 485 / 512 (94.7265625%)
Maximum Difference in Layer 6: 0.3603161
Amount of different weights in Layer 7: 4857 / 5120 (94.86328125%)
Maximum Difference in Layer 7: -0.10776067


In [12]:
res = []
for i in range(1, 61):
    t = 'redundancy_check/base_old_60-{}'.format(str(i).zfill(2))
    s = 'redundancy_check/student_old_60-{}'.format(str(i).zfill(2))
    
    m1 = tf.keras.models.load_model(t)
    m2 = tf.keras.models.load_model(s)
    
    stats = weights_diff_stats(m1, m2)
    res.append(stats)

In [14]:
%matplotlib widget

lines = []
for j in range(len(res[0])):
    line = []
    for i in range(len(res)):
        line.append(res[i][j])
    lines.append(line)

for line in lines:
    plt.plot(line)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [2]:
def epoch_weight_diff(m1_prefix, m2_prefix):
    res = []
    for i in range(1, 61):
        t = '{}-{}'.format(m1_prefix, str(i).zfill(2))
        s = '{}-{}'.format(m2_prefix, str(i).zfill(2))

        m1 = tf.keras.models.load_model(t)
        m2 = tf.keras.models.load_model(s)

        stats = weights_diff_stats(m1, m2)
        res.append(stats)
    
    lines = []
    for j in range(len(res[0])):
        line = []
        for i in range(len(res)):
            line.append(res[i][j])
        lines.append(line)
    return lines

In [17]:
%matplotlib widget

lines = epoch_weight_diff('redundancy_check/v1_teacher_old_60', 'redundancy_check/v1_student_old_60')

for line in lines:
    plt.plot(line)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
%matplotlib widget

lines = epoch_weight_diff('redundancy_check/v2_teacher_old_60', 'redundancy_check/v2_student_old_60')

for line in lines:
    plt.plot(line)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
m1 = tf.keras.models.load_model('redundancy_check/v2_teacher_old_60')
m2 = tf.keras.models.load_model('redundancy_check/v2_student_old_60')
weights_diff(m1, m2)

Amount of different weights in Layer 1: 57 / 324 (17.59259259259259%)
Maximum Difference in Layer 1: 0.027243175
Amount of different weights in Layer 2: 3657 / 11664 (31.352880658436217%)
Maximum Difference in Layer 2: 0.08973491
Amount of different weights in Layer 3: 3433 / 11664 (29.432441700960222%)
Maximum Difference in Layer 3: 0.018975895
Amount of different weights in Layer 4: 3897 / 11664 (33.410493827160494%)
Maximum Difference in Layer 4: 0.0944793
Amount of different weights in Layer 5: 4487 / 11664 (38.468792866941016%)
Maximum Difference in Layer 5: 0.08519952
Amount of different weights in Layer 6: 5702 / 11664 (48.88545953360769%)
Maximum Difference in Layer 6: 0.08907693
Amount of different weights in Layer 7: 82180 / 165888 (49.53944830246913%)
Maximum Difference in Layer 7: 0.07275871
Amount of different weights in Layer 8: 165 / 512 (32.2265625%)
Maximum Difference in Layer 8: 0.024882078
Amount of different weights in Layer 9: 2261 / 5120 (44.16015625%)
Maximum Dif

In [6]:
%matplotlib widget

lines = epoch_weight_diff('new_model/v1/v1_teacher_old_60', 'new_model/v1/v1_student_old_60')

for line in lines:
    plt.plot(line)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
m1 = tf.keras.models.load_model('new_model/v1/v1_teacher_old_60')
m2 = tf.keras.models.load_model('new_model/v1/v1_student_old_60')
weights_diff(m1, m2)

Amount of different weights in Layer 1: 263 / 288 (91.31944444444444%)
Maximum Difference in Layer 1: 0.5270013
Amount of different weights in Layer 2: 30 / 32 (93.75%)
Maximum Difference in Layer 2: 0.41574627
Amount of different weights in Layer 3: 8735 / 9216 (94.78081597222221%)
Maximum Difference in Layer 3: 0.7891418
Amount of different weights in Layer 4: 31 / 32 (96.875%)
Maximum Difference in Layer 4: 0.654193
Amount of different weights in Layer 5: 24504 / 25600 (95.71875%)
Maximum Difference in Layer 5: 0.82350403
Amount of different weights in Layer 6: 31 / 32 (96.875%)
Maximum Difference in Layer 6: 0.22336972
Amount of different weights in Layer 7: 17740 / 18432 (96.24565972222221%)
Maximum Difference in Layer 7: 0.79646343
Amount of different weights in Layer 8: 62 / 64 (96.875%)
Maximum Difference in Layer 8: 0.46779442
Amount of different weights in Layer 9: 35302 / 36864 (95.76280381944444%)
Maximum Difference in Layer 9: 0.82203484
Amount of different weights in Laye